Creating a Unigram Tokenizer on the new sentences (without considering the definitions)
----------------------------------
We added sentences got from `omniglot` inside the diagne's sentences. Since we want to test the relevancy of the new sentences to our translation task, let us create a tokenizer for them. It is done in order to train the T5 model on it and see if we obtain a better performance. 

The process is almost the same as in [processing_4](text_processing4.ipynb) excepted that we will create another custom dataset for the custom transformer model.

Let us import the necessary libraries.

In [1]:
# for creating the tokenizer
from tokenizers import (
    decoders,
    models,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    normalizers
)

# for importing and manipulating the sentences
import pandas as pd
import random

# for loading sentences with the custom dataset
from torch.utils.data import DataLoader

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load dataset and create generator

We will create one tokenizer for both of the French and Wolof corpora because the `T5` model' understand only one embedding layer. So we must create one generator for both of the French and Wolof corpora. 

In [2]:
# load sentences
sentences = pd.read_csv("data/extractions/new_data/ad_sentences.csv")

# initialize a batch size
BATCH_SIZE = 400

# create generators (for the corpora)
def generate_sents():
    
    # recuperate the sentences
    french_sents = sentences['french'].to_list() 
    
    wolof_sents = sentences['wolof'].to_list() 
    
    sents = french_sents + wolof_sents
    
    for i in range(1, len(sents), BATCH_SIZE):
        
        yield sents[i:i+BATCH_SIZE]

#### Initialize the tokenizer

In [3]:
tokenizer = Tokenizer(models.Unigram())

#### Add normalizer

In [4]:
tokenizer.normalizer = normalizers.Replace(" {2,}", " ")

#### Configure the pre-tokenizers

We will use the Metaspace pre-tokenizer which separates the words considering the spaces between them. It will replace the space by a character (by default the underscore "_").

In [5]:
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

#### Initialize the trainers

We will provide all of the necessary special tokens to the Trainer. 

**Notice that a sentence can be a group of words separated by ending marks and not only one group of words. Then we can, for example, tokenize the following sentences**: `<sep>sentence1.sentence2.sentence3<cls>` **or** `<sep>sentence1.<sep>sentence2.<cls>`. **But, the second sentence is composed of two separate groups. Then the two sentences will have different type ids.** 

In [6]:
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]

In [7]:
trainer = trainers.UnigramTrainer(special_tokens=special_tokens, unk_token = "<unk>") # let us take the default vocab size

#### Train the tokenizer

In [8]:
tokenizer.train_from_iterator(generate_sents(), trainer)

Let us print the vocab size.

In [9]:
print(f"Number of tokens: {tokenizer.get_vocab_size()}")

Number of tokens: 3070


#### Initialize the post-processor

We can not need the TemplateProcessor to train our corpora in a Sequence To Sequence model, but we will add it to our tokenizer. We can use it for another type of model. 

In [10]:
# let us recuperate the sep and cls ids
cls_token_id = tokenizer.token_to_id("<cls>")

sep_token_id = tokenizer.token_to_id("<sep>")

print(cls_token_id, sep_token_id)

0 1


In [11]:
# Initialize the post processor
tokenizer.post_process = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)]
)

#### Initialize the decoder

In [12]:
tokenizer.decoder = decoders.Metaspace()

#### Save the tokenizer

In [13]:
tokenizer.save("wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3_2.json")

#### Make a little example

Let us recuperate random sentences from the corpora and tokenize them.

In [14]:
random.seed(200)

french_sentence = random.choice(sentences['french']) + "</s>"

wolof_sentence = random.choice(sentences['wolof']) + "</s>"

In [15]:
# print the french sentence
french_sentence

'Fais sortir tout cheval que tu vois !</s>'

In [16]:
# print the wolof sentence
wolof_sentence

'Waaw nataal bii de ay bunt yu dóomu-taal moo ci nekk. Bunt yi ëe dafa dóoomu-taal waawaw! Am na ay lu mel ni ay fu ngelaw di jaar boo ko xëccee waaw! Mën nga, mën nga ko xëcc, bunt yi waaw. Du benn bunt nag.</s>'

In [17]:
french_encoding = tokenizer.encode(french_sentence)

print("French tokens")
print(french_encoding.tokens)

print("French ids")
print(french_encoding.ids)

French tokens
['▁Fa', 'is', '▁sortir', '▁tout', '▁cheval', '▁que', '▁tu', '▁vois', '▁!', '</s>']
French ids
[261, 311, 739, 147, 568, 28, 46, 154, 23, 6]


In [18]:
wolof_encoding = tokenizer.encode(wolof_sentence)

print("Wolof tokens")
print(wolof_encoding.tokens)

print("Wolof ids")
print(wolof_encoding.ids)

Wolof tokens
['▁Waaw', '▁na', 'taal', '▁bii', '▁de', '▁ay', '▁bunt', '▁yu', '▁dóo', 'mu', '-', 'taal', '▁moo', '▁ci', '▁nekk', '.', '▁Bunt', '▁yi', '▁ëe', '▁da', 'fa', '▁dóo', 'omu', '-', 'taal', '▁waaw', 'aw', '!', '▁Am', '▁na', '▁ay', '▁lu', '▁mel', '▁ni', '▁ay', '▁fu', '▁nge', 'law', '▁di', '▁jaar', '▁boo', '▁ko', '▁xëcc', 'ee', '▁waaw', '!', '▁Më', 'n', '▁nga', ',', '▁mën', '▁nga', '▁ko', '▁xëcc', ',', '▁bunt', '▁yi', '▁waaw', '.', '▁Du', '▁benn', '▁bunt', '▁nag', '.', '</s>']
Wolof ids
[210, 21, 295, 99, 15, 56, 259, 83, 1215, 969, 180, 295, 184, 18, 102, 8, 1054, 74, 540, 228, 317, 1215, 2274, 180, 295, 273, 468, 11, 165, 21, 56, 124, 150, 82, 56, 260, 1260, 1717, 63, 908, 109, 105, 1026, 467, 273, 11, 920, 20, 153, 9, 318, 153, 105, 1026, 9, 259, 74, 273, 8, 290, 122, 259, 66, 8, 6]


#### Creating the T5 custom dataset for the new sentences

We have two possibilities to use the tokenizer for fine-tuning a T5 model. 

- We can use the `PreTrainedTokenizerFast` class for which we will provide the different special tokens.

In [19]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer1 = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

- Or give directly the tokenizer to the `T5TokenizerFast` class.

In [20]:
from transformers import XLNetTokenizerFast

In [21]:
from transformers import T5TokenizerFast

wrapped_tokenizer2 = T5TokenizerFast(
    tokenizer_object=tokenizer
)

Let us give them the sentences that we use as example. 

In [22]:
fr_encoding = wrapped_tokenizer1(french_sentence, max_length=15, padding='max_length', truncation=True)

fr_encoding

{'input_ids': [3, 3, 3, 3, 3, 261, 311, 739, 147, 568, 28, 46, 154, 23, 6], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
wf_encoding = wrapped_tokenizer2(wolof_sentence, max_length=15, padding='max_length', truncation=True)

wf_encoding

{'input_ids': [210, 21, 295, 99, 15, 56, 259, 83, 1215, 969, 180, 295, 184, 18, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Let us decode the wolof sentence.

In [24]:
wrapped_tokenizer1.decode(wf_encoding.input_ids, skip_special_tokens=True)

'Waaw nataal bii de ay bunt yu dóomu-taal moo ci nekk'

We can see that the `T5Tokenizer` add padding to the right side of the sequence while the `PretrainedTokenizer` add the padding to the left side. We can change the padding side from the settings. But, for the next steps, let us directly use the `T5Tokenizer`.

**Note that we can augment the sentences when generating them like we did when using the `GPT2 model`.** See the following notebook, [augmentation](text_augmentation.ipynb), for discussion on the augmentation method that we will use. And for a more clear explanation of the augmentation methods in NLP tasks and training, look at the following article from the web [augment_or_not](https://direct.mit.edu/coli/article/48/1/5/108844/To-Augment-or-Not-to-Augment-A-Comparative-Study).

Let us verify, before creating the custom dataset, the max length that we can get from the corpora' tokens without considering the augmentation.

In [29]:
max_len = 0

for sent in sentences['french'].to_list() + sentences['wolof'].to_list():
    
    len_ids = len(wrapped_tokenizer2(sent).input_ids)
    
    if len_ids > max_len:
        
        max_len = len_ids

        

In [30]:
# let us print the max lengths
max_len

193

We find a maximum length of **193** tokens. But considering the augmentation we can obtain more than 42 tokens because it will add modifications on the words and then it can recognize only parts of them and divide them in multiple other tokens. Let us add to the max length the fifth of it. 

In [31]:
max_len += max_len // 5

max_len

231

It is time to create our custom dataset.

Signature:
```python
class T5SentenceDataset(Dataset):

    def __init__(
        self,
        data_path: str, 
        tokenizer: PreTrainedTokenizerFast
        corpus_1: str = "french",
        corpus_2: str = "wolof",
        max_len: int = 231,
        cp1_truncation: bool = False,
        cp2_truncation: bool = False,
        file_sep: str = ",",
        cp1_transformer: Union[TransformerSequences, None] = None,
        cp2_transformer: Union[TransformerSequences, None] = None,
        **kwargs):

        pass
```